### download the last month of the CDS raw forecasts 

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
import sys
import pathlib
from datetime import datetime
from dateutil.relativedelta import relativedelta
import dateparser

In [44]:
from dask import delayed, compute
from dask.diagnostics import ProgressBar

In [45]:
HOME = pathlib.Path.home()

In [46]:
CWD = pathlib.Path.cwd()

In [47]:
CWD

PosixPath('/home/nicolasf/operational/ICU/development/hotspots/code/ICU_Water_Watch/notebooks/C3S')

### import local functions for the downloading of the C3S forecasts 

In [48]:
sys.path.append('../..')

In [49]:
from ICU_Water_Watch import C3S, domains

### show the predefined domains 

In [50]:
domains.domains

{'Tropical_Pacific': [140, 220, -25, 25],
 'SW_Pacific': [172.5, 190, -22.5, -12],
 'Fiji': [175, 183, -21, -15],
 'NZ': [161, 181, -50, -30],
 'Pacific': [140, 240, -50, 25],
 'C3S_download': [100, 240, -50, 30],
 'Water_Watch': [125, 240, -35, 25]}

### provider (always CDS for now)

In [51]:
provider = 'CDS'

### variable name

In [52]:
var_name = 'tprate'

### list of GCMs 

In [53]:
list_GCMs = ['ECMWF','UKMO','METEO_FRANCE','CMCC','DWD', 'NCEP', 'JMA', 'ECCC_CanCM4i', 'ECCC_GEM_NEMO']

### mapping GCM to systems 

In [54]:
from collections import OrderedDict as OD

In [55]:
dict_systems = OD(
                ECMWF=5, # SEAS5, hindcast complete 
                UKMO=601, # note: only for system 14 is hindcast complete
                METEO_FRANCE=8, # note: only for system 7 is hindcast complete
                DWD=21, # GCFS2.1, hindcast complete 
                CMCC=35, # SPSv3.5, hindcast complete 
                NCEP=2,# CFSv2, hindcast complete 
                JMA=3, # CPS2, hindcast complete
                ECCC_GEM_NEMO=2, # GEM-NEMO, hindcast incomplete
                ECCC_CanCM4i=3, # CanCM4i, hindcast incomplete
                )

### lag in months (if need to download older forecasts)

In [56]:
lag = 0 

### get today's date 

In [57]:
date = dateparser.parse('today')

In [58]:
date

datetime.datetime(2022, 3, 14, 10, 54, 50, 304501)

### apply lag 

In [59]:
date = date - relativedelta(months=lag)

In [60]:
print(f"will download the forecasts issued in {date:%B %Y}")

will download the forecasts issued in March 2022


### path to download the realtime forecasts 

In [61]:
gcm_path = pathlib.Path(f'/media/nicolasf/END19101/ICU/data/{provider}/operational/forecasts')

In [62]:
if not(gcm_path.exists()): 
    gcm_path.mkdir(parents=True)

### use dask to parallelize the download of the forecasts 

#### build the queue

In [63]:
downloaded_files = []

for GCM in dict_systems.keys():
    
    if 'ECCC' in GCM:
        
        GCM_API_name = GCM.split('_')[0]
        GCM_path = gcm_path.joinpath(GCM).joinpath(var_name.upper())
        
        print(f"processing ECCC, path should be {str(GCM_path)}")
    
        args = dict(GCM=GCM_API_name,
        varname=var_name,
        year=date.year,
        month=date.month,
        leadtimes=[1, 2, 3, 4, 5, 6],
        opath=GCM_path,
        domain=domains.domains['C3S_download'],
        file_format='netcdf',
        level='surface',
        max_retry=3)
    
    else: 
        
        args = dict(GCM=GCM,
        varname=var_name,
        year=date.year,
        month=date.month,
        leadtimes=[1, 2, 3, 4, 5, 6],
        opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
        domain=domains.domains['C3S_download'],
        file_format='netcdf',
        level='surface',
        max_retry=3)    
    
    downloaded_files.append(delayed(C3S.download)(**args))


processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_GEM_NEMO/TPRATE
processing ECCC, path should be /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_CanCM4i/TPRATE


#### now compute 

In [64]:
with ProgressBar():
    downloaded_files = compute(downloaded_files)

[                                        ] | 0% Completed |  0.0s
attempting to download tprate for GCM METEO_FRANCE, for level surface, year 2022, month 3, in netcdf, attempt 1 of 3


attempting to download tprate for GCM DWD, for level surface, year 2022, month 3, in netcdf, attempt 1 of 3


attempting to download tprate for GCM CMCC, for level surface, year 2022, month 3, in netcdf, attempt 1 of 3


attempting to download tprate for GCM ECCC, for level surface, year 2022, month 3, in netcdf, attempt 1 of 3


attempting to download tprate for GCM JMA, for level surface, year 2022, month 3, in netcdf, attempt 1 of 3

attempting to download tprate for GCM ECMWF, for level surface, year 2022, month 3, in netcdf, attempt 1 of 3



attempting to download tprate for GCM ECCC, for level surface, year 2022, month 3, in netcdf, attempt 1 of 3


attempting to download tprate for GCM UKMO, for level surface, year 2022, month 3, in netcdf, attempt 1 of 3

attempting to download tprate for GCM NC

2022-03-14 10:54:55,526 INFO Welcome to the CDS
2022-03-14 10:54:55,527 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-03-14 10:54:55,531 INFO Welcome to the CDS
2022-03-14 10:54:55,532 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-03-14 10:54:55,540 INFO Welcome to the CDS
2022-03-14 10:54:55,541 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-03-14 10:54:55,544 INFO Welcome to the CDS
2022-03-14 10:54:55,545 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-03-14 10:54:55,549 INFO Welcome to the CDS
2022-03-14 10:54:55,550 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/seasonal-monthly-single-levels
2022-03-14 10:54:55,554 INFO Welcome to the CDS
2022-03-14 10:54:55,555 INFO Welcome to the CDS
2022-03-14 10:

[                                        ] | 0% Completed |  2.3s

2022-03-14 10:54:55,869 INFO Request is completed
2022-03-14 10:54:55,871 INFO Downloading https://download-0006.copernicus-climate.eu/cache-compute-0006/cache/data8/adaptor.mars.external-1647204403.8488767-10872-13-c0015af2-0b10-4880-8981-5827f75c250d.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/NCEP/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03_NCEP.netcdf (12.2M)
2022-03-14 10:54:55,873 INFO Request is completed
2022-03-14 10:54:55,876 INFO Request is completed
2022-03-14 10:54:55,884 INFO Request is completed
2022-03-14 10:54:55,886 INFO Request is completed
2022-03-14 10:54:55,887 INFO Request is completed
2022-03-14 10:54:55,887 INFO Downloading https://download-0014.copernicus-climate.eu/cache-compute-0014/cache/data2/adaptor.mars.external-1647204431.1138828-20452-14-cd07b156-967f-413c-aa56-8066c6b1f295.nc to /media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/DWD/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03_DWD.netcdf (5.4M)
2022-03-14 10

[                                        ] | 0% Completed |  2.4s

...

[###################################     ] | 88% Completed | 14.7s








 93%|████████████████████████████████████████████████████████████████████████████████████████████████▎       | 2.02M/2.18M [00:10<00:00, 450kB/s]

[###################################     ] | 88% Completed | 15.0s








                                                                                                                                                 2022-03-14 10:55:08,522 INFO Download rate 177K/s


[###################################     ] | 88% Completed | 15.3s
/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECCC_GEM_NEMO/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03_ECCC.netcdf downloaded OK

[########################################] | 100% Completed | 15.4s


In [24]:
# for GCM in list_GCMs: 
    
#     f = C3S.download(
#     GCM=GCM,
#     varname=var_name,
#     year=date.year,
#     month=date.month,
#     leadtimes=[1, 2, 3, 4, 5, 6],
#     opath=gcm_path.joinpath(GCM).joinpath(var_name.upper()),
#     domain=domains.domains['C3S_download'],
#     file_format='netcdf',
#     level='surface',
#     max_retry=3,
#     )   
#     lfiles.append(f)

### check dimensions and return the total number of members in the C3S MME 

In [25]:
import xarray as xr

In [26]:
members = []
for f in downloaded_files[0]: 
    dset = xr.open_dataset(f)
    members.append(dset.dims['number'])
    print(dset.dims)
    if 'system' in dset.dims:
        print(dset['system'])
    

Frozen({'longitude': 141, 'latitude': 81, 'number': 51, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 56, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 51, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 50, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 50, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 112, 'time': 5})
Frozen({'longitude': 113, 'latitude': 65, 'number': 140, 'time': 5})
Frozen({'longitude': 141, 'latitude': 81, 'number': 10, 'time': 5, 'system': 2})
<xarray.DataArray 'system' (system: 2)>
array([2, 3], dtype=int32)
Coordinates:
  * system   (system) int32 2 3
Attributes:
    long_name:  system
Frozen({'longitude': 141, 'latitude': 81, 'number': 10, 'time': 5, 'system': 2})
<xarray.DataArray 'system' (system: 2)>
array([2, 3], dtype=int32)
Coordinates:
  * system   (system) int32 2 3
Attributes:
    long_name:  system


In [27]:
import numpy as np 

In [28]:
np.array(members).sum()

530

In [29]:
downloaded_files[0]

[PosixPath('/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/ECMWF/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03_ECMWF.netcdf'),
 PosixPath('/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/UKMO/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03_UKMO.netcdf'),
 PosixPath('/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/METEO_FRANCE/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03_METEO_FRANCE.netcdf'),
 PosixPath('/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/DWD/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03_DWD.netcdf'),
 PosixPath('/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/CMCC/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03_CMCC.netcdf'),
 PosixPath('/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/NCEP/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03_NCEP.netcdf'),
 PosixPath('/media/nicolasf/END19101/ICU/data/CDS/operational/forecasts/JMA/TPRATE/ensemble_seas_forecasts_tprate_from_2022_03